<a href="https://colab.research.google.com/github/tono000/CS114.K11/blob/master/Pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import train_test_split
import random
import time
import cv2
import imutils

In [0]:
from google.colab import drive
drive.mount('/mydrive',True)

In [0]:
path = '/mydrive/My Drive/Dataset_cuoiki'
folder = path
print(folder)

**Cắt hình ảnh sử dụng hog**

In [0]:
from imutils.object_detection import non_max_suppression
from imutils import paths

# Khởi tạo
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

In [0]:
from google.colab.patches import cv2_imshow 
import time

dem=0
start=time.time()
for i in os.listdir(folder):

    # load the image and resize it to (1) reduce detection time
    # and (2) improve detection accuracy
    image = cv2.imread(folder+'/'+i)
    image = imutils.resize(image, width=min(400, image.shape[1]))
    orig = image.copy()

    # detect people in the image
    (rects, weights) = hog.detectMultiScale(image, winStride=(4, 4),
      padding=(8, 8), scale=1.05)

   
    # apply non-maxima suppression to the bounding boxes using a
    # fairly large overlap threshold to try to maintain overlapping
    # boxes that are still people
    rects = np.array([[x, y, x + w, y + h] for (x, y, w, h) in rects])
    pick = non_max_suppression(rects, probs=None, overlapThresh=0)

    # draw the final bounding boxes
    #for (xA, yA, xB, yB) in pick:
    #  cv2.rectangle(image, (xA, yA), (xB, yB), (0, 255, 0), 2)
    dem=dem+1

    if len(pick)!=0:
      (x,y,w,h)=pick[0]
      print('i=',i)
      imgCrop = image[y:h,x:w]
      #imgCrop=cv2.resize(imgCrop,(50,50))
      #imgCrop = cv2.cvtColor(imgCrop, cv2.COLOR_BGR2GRAY)
      cv2_imshow(imgCrop)
      uploaded='/mydrive/My Drive/hog_dataset/'+str(dem)+'.jpg'
      #dem=dem+1
      cv2.imwrite(uploaded,imgCrop)
    # show the output images
    #cv2_imshow(image)
    #cv2.waitKey(0)

end=time.time()

**Instance segmentation sử dụng R-CNN**

In [0]:
# Khởi tạo
model_fol='/mydrive/My Drive/R-CNN instance segmentation'
LABELS = open(model_fol+'/object_detection_classes_coco.txt').read().strip().split("\n")

weightsPath = model_fol+'/frozen_inference_graph.pb'
configPath = model_fol+'/mask_rcnn_inception_v2_coco_2018_01_28.pbtxt'

print("[INFO] loading Mask R-CNN from disk...")
net = cv2.dnn.readNetFromTensorflow(weightsPath, configPath)

In [0]:
from google.colab.patches import cv2_imshow 
dem=0
start1 = time.time()

for file_name in os.listdir(folder):
	net = cv2.dnn.readNetFromTensorflow(weightsPath, configPath)

	# load ảnh
	image = cv2.imread(folder+'/'+file_name)
	image = imutils.resize(image, width=600)	
	(H, W) = image.shape[:2]
	print(file_name)

	# Preprocess tạo một blob làm ảnh đầu vào cho R-CNN
	blob = cv2.dnn.blobFromImage(image, swapRB=True, crop=False)
	net.setInput(blob)
	print('blob ok')

	start = time.time()
	(boxes, masks) = net.forward(["detection_out_final", "detection_masks"])
	print("[INFO] boxes shape: {}".format(boxes.shape))

	end = time.time()
	idxs = np.argsort(boxes[0, 0, :, 2])[::-1]
	mask = None
	roi = None
	coords = None

	for i in range(0, boxes.shape[2]):
		# lấy confidence (probability) để xét xem nó là người bao nhiêu phần trăm
		classID = int(boxes[0, 0, i, 1])
		confidence = boxes[0, 0, i, 2]
		print(confidence)


		if LABELS[classID] != "person":
			continue
		# loại bỏ những đối tượng ko phải là người
		if confidence > 0.5:
			clone = image.copy()
			print('c',confidence)
		
			# tạo ô xung quanh đối tượng
			box = boxes[0, 0, i, 3:7] * np.array([W, H, W, H])
			(startX, startY, endX, endY) = box.astype("int")
			boxW = endX - startX
			boxH = endY - startY

			# gán nhãn cho đối tượng
			mask = masks[i, classID]
			mask = cv2.resize(mask, (boxW, boxH),
				interpolation=cv2.INTER_NEAREST)
			mask = (mask > 0.3)

			# cắt phần ảnh có đói tượng
			roi = clone[startY:endY, startX:endX]


			if 1> 0:
				# extract ra ảnh vào thư mục
				visMask = (mask * 255).astype("uint8")
				instance = cv2.bitwise_and(roi, roi, mask=visMask)

				uploaded='/mydrive/My Drive/hog_RCNN dataset/roi'+str(dem)+'.jpg'
				dem=dem+1
				cv2.imwrite(uploaded,roi)
				
				uploaded='/mydrive/My Drive/hog_RCNN dataset/visMask'+str(dem)+'.jpg'
				cv2.imwrite(uploaded,visMask)
				
				uploaded='/mydrive/My Drive/hog_RCNN dataset/instance'+str(dem)+'.jpg'
				cv2.imwrite(uploaded,instance)
				print(dem)
				break

end1 = time.time()
